<a href="https://colab.research.google.com/github/NSCC-HydroPandas/Canadian-Tidal-Data-Metadat/blob/main/Downloading_Monthly_Means_MultiStation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Set up Enviroment**

In [ ]:
!pip install requests
!pip install pandas
import requests
import pandas as pd
import json
import time
from urllib.parse import urlparse, parse_qs

**2. User Input**

A CSV with stations, start and end dates

In [ ]:
csv_path = (input('Please Provide path to station csv: '))

stations_df = pd.read_csv(csv_path)
stations_df['Station'] = stations_df['Station'].apply(lambda x: f"{x:05d}")
station_list = stations_df['Station'].tolist()

**4. Find station endpoint based on station number provided**

In [ ]:
for index, row in stations_df.iterrows():
    stn_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={row["Station"]}'
    stations_df.loc[index, 'Station_URL'] = stn_url

In [ ]:
for index, row in stations_df.iterrows():
    stn_url = row['Station_URL']
    try:
        df = pd.read_json(stn_url)
        stn_id = df['id'].values[0]
    except:
        
        print(f'Station Not Found for URL: {stn_url}')
        continue
    stations_df.loc[index, 'Station_ID'] = stn_id


**5. Create List of url for API requests & Request Monthly Means using the URL request list From the Public API**

In [ ]:
for index, row in stations_df.iterrows():

    if not pd.isna(row['Station_ID']):
        request_list = []   # <-- Reset for each station

        stn_id = row['Station_ID']
        start_date = int(row['Start year'])
        end_date = int(row['End year'])

        # Build URL list for this station only
        for year in range(start_date, end_date + 1):
            for month in range(1, 13):
                month_str = f"{month:02d}"
                data_url = (
                    f"https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/{stn_id}/stats/calculate-monthly-mean"
                    f"?year={year}&month={month_str}&timeZone=UTC"
                )
                request_list.append(data_url)

        data = pd.DataFrame()

        # Fetch and process data
        for url in request_list:
            parsed = urlparse(url)
            params = parse_qs(parsed.query)
            year = int(params['year'][0])
            month = int(params['month'][0])

            try:
                r = requests.get(url)
                d = r.json()
                df = pd.DataFrame([d])
                df['Station'] = row['Station']
                df['year'] = year
                df['month'] = f"{month:02d}"
                data = pd.concat([data, df], ignore_index=True)

            except Exception as e:
                print(f"No Monthly mean for {year}-{month:02d} due to insufficient data")
                continue

        # Save per-station CSV
        data.to_csv(f'{row["Station"]}_{start_date}-01_{end_date}-12.csv', index=False)